In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import time
import numpy as np
import matplotlib.pyplot as plt
from pympc.dynamical_systems import DTPWASystem, DTAffineSystem, dare, moas_closed_loop_from_orthogonal_domains
from pympc.control import MPCHybridController
from pympc.geometry.polytope import Polytope
import pympc.plot as mpc_plt
from copy import copy
import director.viewerclient as vc
from director.thirdparty import transformations

In [3]:
class LabeledArray:
    
    def __init__(self, axis_labels, array=None):
        self.axis_labels = axis_labels
        if array is None:
            self.array = np.zeros([len(label) for label in axis_labels])
        else:
            self.array = array
        return
    
    def get_indices(self, labels):
        if not isinstance(labels, tuple):
             labels = (labels,)
        return tuple(self.axis_labels[i].index(l) for i, l in enumerate(labels))
    
    def __getitem__(self, labels):
        return self.array[self.get_indices(labels)]
    
    def __setitem__(self, labels, value):
        self.array[self.get_indices(labels)] = value
        return
    
    @property
    def shape(self):
        return self.array.shape

parameters

In [4]:
m = 1.
I = 1.
g = 10.
h = .1
mu = .5
d = 1.
tol = 1.e-4

In [5]:
x_labels = ['qbx','qby','thb',
    'qlfx','qlfy','qrfx','qrfy',
    'qlhx','qlhy','qrhx','qrhy',
    'vbx','vby','omb',
    'vlfx','vlfy','vrfx','vrfy',
    'vlhx','vlhy','vrhx','vrhy']
u_labels = ['flfn','flft','frfn','frft',
    'flhn','flht','frhn','frht',
    'alfx','alfy','arfx','arfy',
    'alhx','alhy','arhx','arhy']
n_x = len(x_labels)
n_u = len(u_labels)

equilibrium state

In [6]:
x_eq = LabeledArray([x_labels])
x_eq['qbx'] = .5
x_eq['qby'] = .5
x_eq['qlfx'] = .7
x_eq['qrfx'] = .3
x_eq['qlhx'] = .8
x_eq['qlhy'] = .6
x_eq['qrhx'] = .2
x_eq['qrhy'] = .6

equilibrium input

In [7]:
u_eq = LabeledArray([u_labels])
u_eq['flfn'] = m*g/2.
u_eq['frfn'] = m*g/2.
u_eq['flhn'] = tol
u_eq['frhn'] = tol

dynamics

In [8]:
A = LabeledArray([x_labels, x_labels])
A['qbx','vbx'] = 1.
A['qby','vby'] = 1.
A['thb','omb'] = 1.
A['qlfx','vlfx'] = 1.
A['qlfy','vlfy'] = 1.
A['qrfx','vrfx'] = 1.
A['qrfy','vrfy'] = 1.
A['qlhx','vlhx'] = 1.
A['qlhy','vlhy'] = 1.
A['qrhx','vrhx'] = 1.
A['qrhy','vrhy'] = 1.

In [9]:
B = LabeledArray([x_labels, u_labels])

# horizontal equilibrium body
B['vbx','flft'] = -1./m
B['vbx','frft'] = -1./m
B['vbx','flhn'] = -1./m
B['vbx','frhn'] = 1./m

# vertical equilibrium body
B['vby','flfn'] = 1./m
B['vby','frfn'] = 1./m
B['vby','flht'] = -1./m
B['vby','frht'] = 1./m

# rotational equilibrium body
B['omb','flfn'] = (x_eq['qlfx'] - x_eq['qbx'])/I
B['omb','flft'] = (x_eq['qlfy'] - x_eq['qby'])/I
B['omb','frfn'] = (x_eq['qrfx'] - x_eq['qbx'])/I
B['omb','frft'] = (x_eq['qrfy'] - x_eq['qby'])/I
B['omb','flhn'] = (x_eq['qlhy'] - x_eq['qby'])/I
B['omb','flht'] = (x_eq['qbx'] - x_eq['qlhx'])/I
B['omb','frhn'] = (x_eq['qby'] - x_eq['qrhy'])/I
B['omb','frht'] = (x_eq['qlhx'] - x_eq['qbx'])/I

# limbs acceleration control
B['vlfx','alfx'] = 1.
B['vlfy','alfy'] = 1.
B['vrfx','arfx'] = 1.
B['vrfy','arfy'] = 1.
B['vlhx','alhx'] = 1.
B['vlhy','alhy'] = 1.
B['vrhx','arhx'] = 1.
B['vrhy','arhy'] = 1.

In [10]:
c = LabeledArray([x_labels])
c['vby'] = -g
c_translated = c.array + A.array.dot(x_eq.array) + B.array.dot(u_eq.array)

In [11]:
S = DTAffineSystem.from_continuous(A.array, B.array, c_translated, h, method='zoh')

limbs displacement from nominal position

In [12]:
box_size = .15
limb_x = ['qlfx','qrfx','qlhx','qrhx']
limb_y = ['qlfy','qrfy','qlhy','qrhy']
lhs = LabeledArray([limb_x + limb_y, x_labels])
rhs = LabeledArray([limb_x + limb_y])
for l in limb_x:
    lhs[l,l] = 1.
    lhs[l,'qbx'] = -1.
    rhs[l] = x_eq[l] - x_eq['qbx']
for l in limb_y:
    lhs[l,l] = 1.
    lhs[l,'qby'] = -1.
    rhs[l] = x_eq[l] - x_eq['qby']
X = Polytope(lhs.array, rhs.array + np.ones(rhs.shape)*box_size)
X.add_facets(-lhs.array, np.ones(rhs.array.shape)*box_size - rhs.array)

In [13]:
# body postion
X.add_bounds(x_eq['qbx']-box_size, x_eq['qbx']+box_size, [x_labels.index('qbx')])
X.add_bounds(x_eq['qby']-box_size, x_eq['qby']+box_size, [x_labels.index('qby')])
# body angle
X.add_bounds(-.1, .1, [x_labels.index('thb')])
# body linear velocity
X.add_bounds(-.5, .5, [x_labels.index('vbx')])
X.add_bounds(-.5, .5, [x_labels.index('vby')])
# body angular velocity
X.add_bounds(-.1, .1, [x_labels.index('omb')])
# limbs linear velocity
X.add_bounds(np.vstack([-1.]*8), np.vstack([1.]*8), range(14,n_x))

input limits

In [14]:
limb_labels = ['lf','rf','lh','rh']

# upper bound friction force
lhs = LabeledArray([limb_labels, u_labels])
rhs = LabeledArray([limb_labels])
for l in limb_labels:
    lhs[l,'f'+l+'t'] = 1.
    lhs[l,'f'+l+'n'] = -mu
U = Polytope(lhs.array, rhs.array)

# lower bound friction force
lhs = LabeledArray([limb_labels, u_labels])
rhs = LabeledArray([limb_labels])
for l in limb_labels:
    lhs[l,'f'+l+'t'] = -1.
    lhs[l,'f'+l+'n'] = -mu
U.add_facets(lhs.array, rhs.array)

# normal force bounds 
for l in ['flfn','frfn','flhn','frhn']:
    i = u_labels.index(l)
    U.add_bounds(-tol, 2*m*g, [i])

# acceleration bounds
a_min = - 1.
a_max = - a_min
for l in ['alfx','alfy','arfx','arfy','alhx','alhy','arhx','arhy']:
    i = u_labels.index(l)
    U.add_bounds(a_min, a_max, [i])

In [15]:
def add_mode_constraints(mode, X, U):
    if 'lf' in mode:
        X.add_bounds(-tol, tol, [x_labels.index('qlfy')])
        X.add_bounds(-tol, tol, [x_labels.index('vlfx')])
    else:
        X.add_lower_bounds(tol, [x_labels.index('qlfy')])
        U.add_upper_bounds(2.*tol, [u_labels.index('flfn')])
    if 'rf' in mode:
        X.add_bounds(-tol, tol, [x_labels.index('qrfy')])
        X.add_bounds(-tol, tol, [x_labels.index('vrfx')])
    else:
        X.add_lower_bounds(tol, [x_labels.index('qrfy')])
        U.add_upper_bounds(2.*tol, [u_labels.index('frfn')])
    if 'lh' in mode:
        X.add_bounds(d-tol, d+tol, [x_labels.index('qlhx')])
        X.add_bounds(-tol, tol, [x_labels.index('vlhy')])
    else:
        X.add_upper_bounds(d-tol, [x_labels.index('qlhx')])
        U.add_upper_bounds(2.*tol, [u_labels.index('flhn')])
    if 'rh' in mode:
        X.add_bounds(-tol, tol, [x_labels.index('qrhx')])
        X.add_bounds(-tol, tol, [x_labels.index('vrhy')])
    else:
        X.add_lower_bounds(tol, [x_labels.index('qrhx')])
        U.add_upper_bounds(2.*tol, [u_labels.index('frhn')])
    return X, U

In [16]:
modes = [
        ['lf','rf'],
        ['lf','rf','lh'],
        ['lf','rf','rh'],
        ['lf'],
        ['lf','lh'],
        ['lf','rh'],
        ['rf'],
        ['rf','lh'],
        ['rf','rh'],
        ]
S_list = [S]*len(modes)

In [17]:
X_list = []
U_list = []
for mode in modes:
    X_mode = copy(X)
    U_mode = copy(U)
    X_mode, U_mode = add_mode_constraints(mode, X_mode, U_mode)
    lhs_X = X_mode.b - X_mode.A.dot(np.reshape(x_eq.array, (n_x,1)))
    lhs_U = U_mode.b - U_mode.A.dot(np.reshape(u_eq.array, (n_u,1)))
    X_mode = Polytope(X_mode.A, lhs_X)
    U_mode = Polytope(U_mode.A, lhs_U)
    X_mode.assemble()
    U_mode.assemble()
    X_list.append(X_mode)
    U_list.append(U_mode)

In [18]:
pwa_sys = DTPWASystem.from_orthogonal_domains(S_list, X_list, U_list)

In [26]:
N = 10
Q = LabeledArray([x_labels, x_labels])
for l in x_labels:
    Q[l,l] = 1.
for l in limb_x:
    Q[l,'qbx'] = -1.
for l in limb_y:
    Q[l,'qby'] = -1.
Q = Q.array.T.dot(Q.array)
R = np.eye(n_u)/10.
objective_norm = 'two'

In [27]:
# terminal set and cost
P, K = dare(
    S_list[0].A,
    S_list[0].B,
    Q,
    R
    )
#X_N = moas_closed_loop_from_orthogonal_domains(S_list[0].A,S_list[0].B,K,X_list[0],U_list[0])
ub = np.ones((n_x,1))*tol/10.
lb = -ub
X_N = Polytope.from_bounds(lb, ub)
X_N.assemble()
X_N = X_list[0]

In [28]:
# hybrid controller
controller = MPCHybridController(pwa_sys, N, objective_norm, Q, R, P, X_N)

In [29]:
N_sim = 10
terminal_mode = 0
x0 = LabeledArray([x_labels])
x0['vbx'] = .5
x0['vlhx'] = .0
u = []
x = [np.reshape(x0.array, (n_x,1))]
u_ws = None
x_ws = None
ss_ws = None
for k in range(N_sim):
    u_k, x_k, ss_k = controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    print('Time step ' + str(k) + ': ' + str(ss_k) + '.')
    #u_ws = u_k[1:] + [K.dot(x_k[-1])]
    u_ws = u_k[1:] + [np.zeros((n_u,1))]
    #x_ws = x_k[1:] + [pwa_sys.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    x_ws = x_k[1:] + [np.zeros((n_x,1))]
    ss_ws = ss_k[1:] + (terminal_mode,)
    x_next = pwa_sys.simulate(x[k], [u_k[0]])[0][1]
    #print 'Planner-simulator missmatch:', np.linalg.norm(x_next - x_k[1])
    u.append(u_k[0])
    x.append(x_next)

Time step 0: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 1: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 2: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 3: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 4: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 5: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 6: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 7: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 8: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).
Time step 9: (0, 0, 0, 0, 0, 0, 0, 0, 0, 0).


In [ ]:
mpc_plt.input_sequence(u, h)
plt.show()
mpc_plt.state_trajectory(x, h)
plt.show()

## Visualizer

In [23]:
from pympc.models.box_atlas_visualizer import BoxAtlasVisualizer

In [24]:
left_wall = Polytope.from_bounds(np.array([[1.],[0.]]), np.array([[1.01],[1.]]))
left_wall.assemble()
right_wall = Polytope.from_bounds(np.array([[-.01],[0.]]), np.array([[0.],[1.]]))
right_wall.assemble()
floor = Polytope.from_bounds(np.array([[0.],[-.01]]), np.array([[1.],[0.]]))
floor.assemble()
walls = [left_wall, right_wall, floor]

vis = BoxAtlasVisualizer(x_eq, walls)

In [30]:
x_k = LabeledArray([x_labels], x[0])
vis.visualize(x_k)
time.sleep(10 * h)
for k in range(N_sim):
    x_k = LabeledArray([x_labels], x[k])
    vis.visualize(x_k)
    time.sleep(h)

In [ ]:
x_test = LabeledArray([x_labels], x[9])
x_test['qrhx']

In [ ]:
x[1]